In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf

In [106]:
from tensorflow.keras.utils import pad_sequences, to_categorical

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [114]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM

In [124]:
from tensorflow.keras.models import Model

In [18]:
import re

In [45]:
from unidecode import unidecode

In [46]:
import string

In [10]:
arquivo = open("../../../dados/dialogs.txt", "r")

In [11]:
textos = arquivo.readlines()

In [12]:
arquivo.close()

In [31]:
def limpar_texto( texto ):
    texto = texto.replace("\n", "")
    texto = re.sub(r'(?<! )\?', ' ?', texto)
    return texto
    

In [32]:
textos_limpos = [limpar_texto(texto) for texto in textos]

In [35]:
textos_limpos_splited = [texto.split("\t") for texto in textos_limpos]

In [40]:
df = pd.DataFrame(textos_limpos_splited, columns=['perguntas', 'respostas'])

In [41]:
df

,perguntas,respostas
0,"hi, how are you doing ?",i'm fine. how about yourself ?
1,i'm fine. how about yourself ?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been ?
3,no problem. so how have you been ?,i've been great. what about you ?
4,i've been great. what about you ?,i've been good. i'm in school right now.
...,...,...
3720,that's a good question. maybe it's not old age.,are you right-handed ?
3721,are you right-handed ?,yes. all my life.
3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [42]:
contraction_dict = {
    "aren't": "are not", "can't": "can not", "could've": "could have",
    "couldn't": "could not", "daren't": "dare not", "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "he's": "he is",
    "how'd": "how had", "how're": "how are", "how's": "how is",
    "how've": "how have", "i'd": "i had", "i'm": "i am",
    "i've": "i have", "isn't": "is+ not", "it's": "it is",
    "might've": "might have", "mightn't": "might not", "must've": "must have",
    "mustn't": "must not", "needn't": "need not", "oughtn't": "ought not",
    "shan't": "shall not", "she'd": "she had", "she's": "she is",
    "should've": "should have", "shouldn't": "should not", "that'd": "that had",
    "thats's": "that is", "there'd": "there had", "there's": "there is",
    "they'd": "they had", "they're": "you are", "they've": "they have",
    "wasn't": "was+ not", "we'd": "we had", "we're": "we are",
    "we've": "we have", "weren't": "were not", "what'd": "what had",
    "what're": "what are", "what's": "what is", "what've": "what have",
    "when'd": "when had", "when're": "when are", "when's": "when is",
    "when've": "when have", "where'd": "where had", "where're": "where are",
    "where's": "where is", "where've": "where have", "who'd": "who had",
    "who're": "who are", "who's": "who is", "who've": "who have",
    "why'd": "why had", "why're": "why are", "why's": "why is",
    "why've": "why have", "would've": "would have", "wouldn't": "would not",
    "you're": "you are", "you've": "you have", "'cause": "because", 
    "ain't": "is not", "aren't": "are not", "can't": "cannot", 
    "could've": "could have", "he's": "he is", "how'll": "how will",
    "i'll": "i will", "it'll": "it will", "it's": "it is", 
    "she'll": "she will", "she's": "she is", "that'll": "that will",
    "there'll": "there will", "they'll": "they will", "they're": "they are",
    "we'll": "we will", "we're": "we are", "what'll": "what will",
    "when'll": "when will", "where'll": "where will", "who'll": "who will",
    "yo're": "you are", "you'll": "you will"
}

In [64]:
pontuacoes_remover = string.punctuation
pontuacoes_remover = pontuacoes_remover.replace("?", "")

def limpar_trocar_contracoes_tags( texto ):
    global pontuacoes_remover
    novas_palavras = []
    mascara = str.maketrans("\n\r\t", "   ", pontuacoes_remover)
    texto_minusculo = texto.lower()
    lista_palavras = texto_minusculo.split(" ")
    for palavra in lista_palavras:
        if palavra in contraction_dict:
            palavra = contraction_dict[palavra]
        novas_palavras.append(palavra)
    novo_texto =  " ".join(novas_palavras) 
    texto_limpo = novo_texto.translate(mascara)
    texto_limpo = "<START> " + unidecode(texto_limpo) + " <END>"
    return texto_limpo

In [65]:
df['perguntas_limpas'] = df['perguntas'].apply(limpar_trocar_contracoes_tags)
df['respostas_limpas'] = df['respostas'].apply(limpar_trocar_contracoes_tags)

In [66]:
df

,perguntas,respostas,perguntas_limpas,respostas_limpas
0,"hi, how are you doing ?",i'm fine. how about yourself ?,<START> hi how are you doing ? <END>,<START> i am fine how about yourself ? <END>
1,i'm fine. how about yourself ?,i'm pretty good. thanks for asking.,<START> i am fine how about yourself ? <END>,<START> i am pretty good thanks for asking <END>
2,i'm pretty good. thanks for asking.,no problem. so how have you been ?,<START> i am pretty good thanks for asking <END>,<START> no problem so how have you been ? <END>
3,no problem. so how have you been ?,i've been great. what about you ?,<START> no problem so how have you been ? <END>,<START> i have been great what about you ? <END>
4,i've been great. what about you ?,i've been good. i'm in school right now.,<START> i have been great what about you ? <END>,<START> i have been good i am in school right ...
...,...,...,...,...
3720,that's a good question. maybe it's not old age.,are you right-handed ?,<START> thats a good question maybe it is not ...,<START> are you righthanded ? <END>
3721,are you right-handed ?,yes. all my life.,<START> are you righthanded ? <END>,<START> yes all my life <END>
3722,yes. all my life.,you're wearing out your right hand. stop using...,<START> yes all my life <END>,<START> you are wearing out your right hand st...
3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.,<START> you are wearing out your right hand st...,<START> but i do all my writing with my right ...


In [152]:
tokenizer = Tokenizer(oov_token='<OOV>', filters="",)

In [153]:
tokenizer.fit_on_texts(df['perguntas_limpas'] + df['respostas_limpas'])

In [171]:
VOCAB_SIZE = len(tokenizer.word_index) + 1

In [172]:
perguntas_sequences = tokenizer.texts_to_sequences(df['perguntas_limpas'])

In [173]:
respostas_sequences = tokenizer.texts_to_sequences(df['respostas_limpas'])

In [174]:
maximo_palavras_perguntas = max([len(sequence) for sequence in (perguntas_sequences)])

In [175]:
maximo_palavras_respostas = max([len(sequence) for sequence in (respostas_sequences)])

In [176]:
decoder_output_sequences = [ sequence[1:] for sequence in respostas_sequences ]

In [177]:
encoder_input_data = pad_sequences(perguntas_sequences, maxlen=maximo_palavras_perguntas, padding='post')

In [178]:
decoder_input_data = pad_sequences(respostas_sequences, maxlen=maximo_palavras_respostas, padding='post')

In [179]:
decoder_output_data = pad_sequences(decoder_output_sequences, maxlen=maximo_palavras_respostas, padding='post')

In [180]:
decoder_output_data = to_categorical(decoder_output_data, VOCAB_SIZE)

In [181]:
# criar modelo de entrada do encoder
encoder_input = Input(shape=(maximo_palavras_perguntas, ))
encoder_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(encoder_input)
encoder_outputs, state_h, state_c = LSTM(200, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

In [182]:
# criar modelo de entrada do decoder
decoder_input = Input(shape=(maximo_palavras_respostas, ))
decoder_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(decoder_input)
decoder_lstm = LSTM(200, return_state=True, return_sequences=True)

In [183]:
# criar modelo de saida do decoder
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE, activation="softmax")
output = decoder_dense(decoder_output)

In [184]:
model = Model([encoder_input, decoder_input], output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), 
              loss="categorical_crossentropy",
             metrics=['accuracy'])

In [185]:
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)    │ (None, 23)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_10 (InputLayer)   │ (None, 23)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_9 (Embedding)       │ (None, 23, 200)           │         500,000 │ input_layer_9[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_9 (NotEqual)        │ (None, 23)                │               0 │ input_layer_9[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_10 (Embedding)      │ (None, 23, 200)           │         500,000 │ input_layer_10[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_9 (LSTM)                 │ [(None, 200), (None,      │         320,800 │ embedding_9[0][0],         │
│                               │ 200), (None, 200)]        │                 │ not_equal_9[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_10 (LSTM)                │ [(None, 23, 200), (None,  │         320,800 │ embedding_10[0][0],        │
│                               │ 200), (None, 200)]        │                 │ lstm_9[0][1], lstm_9[0][2] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 23, 2500)          │         502,500 │ lstm_10[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,144,100 (8.18 MB)

 Trainable params: 2,144,100 (8.18 MB)

 Non-trainable params: 0 (0.00 B)

In [186]:
decoder_output_data.shape

(3725, 23, 2500)

In [187]:
history = model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size=32, epochs=200)

Epoch 1/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 109ms/step - accuracy: 0.5542 - loss: 6.2340
Epoch 2/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.0983 - loss: 4.7664
Epoch 3/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.0990 - loss: 4.6005
Epoch 4/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - accuracy: 0.1003 - loss: 4.4854
Epoch 5/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.1047 - loss: 4.3822
Epoch 6/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.1081 - loss: 4.3305
Epoch 7/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.1106 - loss: 4.2480
Epoch 8/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - accuracy: 0.1141 - loss: 4.2003
Epoch 9/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.1169 - loss: 4.1115
Epoch 10/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.1213 - loss: 4.0414
Epoch 11/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.1222 - loss: 3.9932
Epoch 12

117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.1799 - loss: 2.5402
Epoch 67/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.1823 - loss: 2.5355
Epoch 68/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.1811 - loss: 2.5217
Epoch 69/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - accuracy: 0.1825 - loss: 2.4857
Epoch 70/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.1860 - loss: 2.4683
Epoch 71/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.1861 - loss: 2.4386
Epoch 72/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.1897 - loss: 2.4288
Epoch 73/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.1885 - loss: 2.4065
Epoch 74/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.1882 - loss: 2.3763
Epoch 75/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.1886 - loss: 2.3552
Epoch 76/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 13s 107ms/step - accuracy: 0.1925 - loss: 2.3557
Epoch 77/200

117/117 ━━━━━━━━━━━━━━━━━━━━ 13s 108ms/step - accuracy: 0.2762 - loss: 1.3798
Epoch 132/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - accuracy: 0.2771 - loss: 1.3561
Epoch 133/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 106ms/step - accuracy: 0.2785 - loss: 1.3387
Epoch 134/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.2790 - loss: 1.3103
Epoch 135/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.2820 - loss: 1.2974
Epoch 136/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.2825 - loss: 1.2841
Epoch 137/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.2841 - loss: 1.2765
Epoch 138/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.2851 - loss: 1.2501
Epoch 139/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - accuracy: 0.2896 - loss: 1.2285
Epoch 140/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.2892 - loss: 1.2148
Epoch 141/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 13s 109ms/step - accuracy: 0.2908 - loss: 1.1909
Ep

117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.3724 - loss: 0.3736
Epoch 197/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 96ms/step - accuracy: 0.3695 - loss: 0.3694
Epoch 198/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 93ms/step - accuracy: 0.3742 - loss: 0.3563
Epoch 199/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.3743 - loss: 0.3543
Epoch 200/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.3749 - loss: 0.3373
